# 📊 Análisis Exploratorio de Datos (EDA) - Tutorial Paso a Paso

## 🎯 Objetivos de este Tutorial

En este notebook aprenderás a realizar un **Análisis Exploratorio de Datos (EDA)** completo usando Python. Al finalizar, sabrás:

- ✅ Cargar y examinar datasets
- ✅ Limpiar y preparar datos
- ✅ Crear visualizaciones informativas
- ✅ Identificar patrones y tendencias
- ✅ Detectar valores atípicos (outliers)
- ✅ Analizar correlaciones entre variables

---

## 📚 ¿Qué es el EDA?

El **Análisis Exploratorio de Datos** es el proceso de investigar un dataset para:
- Entender su estructura y contenido
- Identificar patrones, tendencias y anomalías
- Generar hipótesis sobre los datos
- Decidir qué análisis más profundos realizar

Es como ser un detective de datos! 🕵️‍♂️

## 🔧 Paso 1: Importar las Librerías Necesarias

Comenzamos importando las herramientas que necesitaremos:

In [ ]:
!uv pip install seaborn
!uv pip install plotly


In [ ]:
# Librerías para manipulación de datos
import pandas as pd
import numpy as np

# Librerías para visualización
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Configuración de estilo
plt.style.use('default')
sns.set_palette("husl")

# Configurar matplotlib para mostrar gráficos en español
plt.rcParams['font.size'] = 12
plt.rcParams['figure.figsize'] = (10, 6)

# Suprimir warnings innecesarios
import warnings
warnings.filterwarnings('ignore')

print("✅ ¡Librerías importadas exitosamente!")
print(f"📦 Pandas versión: {pd.__version__}")
print(f"🔢 NumPy versión: {np.__version__}")

## 📂 Paso 2: Cargar los Datos

Para este tutorial, crearemos un dataset de ejemplo que simula datos de ventas de una tienda online. En un caso real, cargarías tus datos desde un archivo CSV, Excel, base de datos, etc.

In [ ]:
# Crear un dataset de ejemplo - Ventas de tienda online
np.random.seed(42)  # Para reproducibilidad

# Generar datos simulados
n_registros = 1000

# Fechas
fechas = pd.date_range(start='2023-01-01', end='2023-12-31', periods=n_registros)

# Categorías de productos
categorias = ['Electrónicos', 'Ropa', 'Hogar', 'Deportes', 'Libros']
categoria = np.random.choice(categorias, n_registros)

# Regiones
regiones = ['Norte', 'Sur', 'Este', 'Oeste', 'Centro']
region = np.random.choice(regiones, n_registros)

# Precios (con distribución realista)
precio_base = {'Electrónicos': 500, 'Ropa': 50, 'Hogar': 200, 'Deportes': 100, 'Libros': 25}
precio = [precio_base[cat] * (0.5 + np.random.random()) for cat in categoria]

# Cantidad vendida
cantidad = np.random.randint(1, 11, n_registros)

# Ingresos totales
ingresos = [p * c for p, c in zip(precio, cantidad)]

# Edad del cliente
edad = np.random.normal(35, 12, n_registros).astype(int)
edad = np.clip(edad, 18, 80)  # Limitar entre 18 y 80 años

# Método de pago
metodo_pago = np.random.choice(['Tarjeta', 'Efectivo', 'PayPal', 'Transferencia'], n_registros)

# Crear el DataFrame
datos = pd.DataFrame({
    'fecha': fechas,
    'categoria': categoria,
    'region': region,
    'precio_unitario': precio,
    'cantidad': cantidad,
    'ingresos_totales': ingresos,
    'edad_cliente': edad,
    'metodo_pago': metodo_pago
})

# Agregar algunos valores nulos para hacer más realista
indices_nulos = np.random.choice(datos.index, size=20, replace=False)
datos.loc[indices_nulos[:10], 'edad_cliente'] = np.nan
datos.loc[indices_nulos[10:], 'metodo_pago'] = np.nan

print(f"✅ Dataset creado exitosamente con {len(datos):,} registros")
print(f"📅 Período: {datos['fecha'].min().strftime('%Y-%m-%d')} a {datos['fecha'].max().strftime('%Y-%m-%d')}")

## 👀 Paso 3: Primera Exploración - Conocer los Datos

Siempre comenzamos con una exploración básica para entender qué tenemos:

In [ ]:
# Ver las primeras filas
print("🔍 PRIMERAS 5 FILAS DEL DATASET:")
print("=" * 50)
display(datos.head())

print("\n📏 FORMA DEL DATASET:")
print(f"Filas: {datos.shape[0]:,}")
print(f"Columnas: {datos.shape[1]}")

print("\n📋 NOMBRES Y TIPOS DE COLUMNAS:")
print("=" * 30)
display(datos.dtypes)

In [ ]:
# Información general del dataset
print("📊 INFORMACIÓN GENERAL DEL DATASET:")
print("=" * 40)
datos.info()

print("\n🔍 ÚLTIMAS 3 FILAS:")
print("=" * 20)
display(datos.tail(3))

## 🧹 Paso 4: Limpieza y Calidad de Datos

Antes de analizar, debemos verificar la calidad de nuestros datos:

In [ ]:
# Verificar valores nulos
print("❌ VALORES NULOS POR COLUMNA:")
print("=" * 30)
valores_nulos = datos.isnull().sum()
porcentaje_nulos = (valores_nulos / len(datos)) * 100

resumen_nulos = pd.DataFrame({
    'Valores_Nulos': valores_nulos,
    'Porcentaje': porcentaje_nulos.round(2)
})

display(resumen_nulos[resumen_nulos['Valores_Nulos'] > 0])

# Verificar duplicados
duplicados = datos.duplicated().sum()
print(f"\n🔄 FILAS DUPLICADAS: {duplicados}")

if duplicados > 0:
    print("⚠️  Se encontraron filas duplicadas. Considera eliminarlas.")
else:
    print("✅ No hay filas duplicadas.")

In [ ]:
# Tratamiento de valores nulos (ejemplo)
print("🔧 TRATAMIENTO DE VALORES NULOS:")
print("=" * 35)

# Para edad: llenar con la mediana
mediana_edad = datos['edad_cliente'].median()
datos['edad_cliente'].fillna(mediana_edad, inplace=True)
print(f"✅ Edad nula llenada con mediana: {mediana_edad:.0f} años")

# Para método de pago: llenar con el más frecuente
metodo_frecuente = datos['metodo_pago'].mode()[0]
datos['metodo_pago'].fillna(metodo_frecuente, inplace=True)
print(f"✅ Método de pago nulo llenado con: {metodo_frecuente}")

# Verificar que no quedan nulos
nulos_restantes = datos.isnull().sum().sum()
print(f"\n🎯 Valores nulos restantes: {nulos_restantes}")

## 📈 Paso 5: Estadísticas Descriptivas

Ahora exploramos las características estadísticas de nuestros datos:

In [ ]:
# Estadísticas para variables numéricas
print("📊 ESTADÍSTICAS DESCRIPTIVAS - VARIABLES NUMÉRICAS:")
print("=" * 55)
display(datos.describe().round(2))

In [ ]:
# Estadísticas para variables categóricas
print("📝 ESTADÍSTICAS DESCRIPTIVAS - VARIABLES CATEGÓRICAS:")
print("=" * 55)

variables_categoricas = ['categoria', 'region', 'metodo_pago']

for variable in variables_categoricas:
    print(f"\n🏷️  {variable.upper()}:")
    conteos = datos[variable].value_counts()
    porcentajes = (datos[variable].value_counts(normalize=True) * 100).round(1)
    
    resumen = pd.DataFrame({
        'Cantidad': conteos,
        'Porcentaje': porcentajes
    })
    
    display(resumen)

## 📊 Paso 6: Visualizaciones Univariadas

Exploramos cada variable por separado para entender sus distribuciones:

In [ ]:
# Histogramas para variables numéricas
variables_numericas = ['precio_unitario', 'cantidad', 'ingresos_totales', 'edad_cliente']

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('📊 Distribución de Variables Numéricas', fontsize=16, fontweight='bold')

for i, variable in enumerate(variables_numericas):
    ax = axes[i//2, i%2]
    
    # Histograma
    datos[variable].hist(bins=30, alpha=0.7, color='skyblue', edgecolor='black', ax=ax)
    
    # Línea de la media
    media = datos[variable].mean()
    ax.axvline(media, color='red', linestyle='--', linewidth=2, label=f'Media: {media:.1f}')
    
    # Configuración
    ax.set_title(f'{variable.replace("_", " ").title()}', fontweight='bold')
    ax.set_xlabel(variable.replace('_', ' ').title())
    ax.set_ylabel('Frecuencia')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Interpretaciones
print("🔍 INTERPRETACIONES:")
print("=" * 20)
print("• Precio unitario: Muestra la distribución de precios por categoría")
print("• Cantidad: La mayoría de compras son de pocas unidades")
print("• Ingresos totales: Distribución sesgada hacia valores más bajos")
print("• Edad cliente: Distribución aproximadamente normal centrada en ~35 años")

In [ ]:
# Gráficos de barras para variables categóricas
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
fig.suptitle('📊 Distribución de Variables Categóricas', fontsize=16, fontweight='bold')

variables_cat = ['categoria', 'region', 'metodo_pago']
colores = ['lightcoral', 'lightgreen', 'lightblue']

for i, (variable, color) in enumerate(zip(variables_cat, colores)):
    conteos = datos[variable].value_counts()
    
    # Gráfico de barras
    conteos.plot(kind='bar', ax=axes[i], color=color, edgecolor='black', alpha=0.8)
    
    # Configuración
    axes[i].set_title(f'{variable.replace("_", " ").title()}', fontweight='bold')
    axes[i].set_xlabel('')
    axes[i].set_ylabel('Frecuencia')
    axes[i].tick_params(axis='x', rotation=45)
    axes[i].grid(True, alpha=0.3)
    
    # Agregar valores en las barras
    for j, v in enumerate(conteos.values):
        axes[i].text(j, v + 5, str(v), ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

## 🔍 Paso 7: Análisis Bivariado

Ahora exploramos las relaciones entre pares de variables:

In [ ]:
# Matriz de correlación
print("🔗 MATRIZ DE CORRELACIÓN:")
print("=" * 25)

# Seleccionar solo variables numéricas
datos_numericos = datos.select_dtypes(include=[np.number])
correlaciones = datos_numericos.corr()

# Crear heatmap
plt.figure(figsize=(10, 8))
mask = np.triu(np.ones_like(correlaciones, dtype=bool))  # Máscara para mostrar solo la mitad

sns.heatmap(correlaciones, 
            mask=mask,
            annot=True, 
            cmap='RdYlBu_r', 
            center=0,
            square=True,
            fmt='.2f',
            cbar_kws={'label': 'Coeficiente de Correlación'})

plt.title('🔗 Matriz de Correlación entre Variables Numéricas', fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Mostrar correlaciones más fuertes
print("\n🎯 CORRELACIONES MÁS FUERTES (|r| > 0.3):")
print("=" * 45)
correlaciones_fuertes = []
for i in range(len(correlaciones.columns)):
    for j in range(i+1, len(correlaciones.columns)):
        var1 = correlaciones.columns[i]
        var2 = correlaciones.columns[j]
        corr = correlaciones.iloc[i, j]
        if abs(corr) > 0.3:
            correlaciones_fuertes.append((var1, var2, corr))

for var1, var2, corr in sorted(correlaciones_fuertes, key=lambda x: abs(x[2]), reverse=True):
    direccion = "positiva" if corr > 0 else "negativa"
    print(f"• {var1} ↔ {var2}: {corr:.3f} ({direccion})")

In [ ]:
# Scatter plots para relaciones interesantes
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Precio vs Ingresos (coloreado por categoría)
for categoria in datos['categoria'].unique():
    mask = datos['categoria'] == categoria
    axes[0].scatter(datos[mask]['precio_unitario'], 
                   datos[mask]['ingresos_totales'],
                   label=categoria, alpha=0.6, s=50)

axes[0].set_xlabel('Precio Unitario')
axes[0].set_ylabel('Ingresos Totales')
axes[0].set_title('💰 Precio vs Ingresos por Categoría')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Edad vs Ingresos
axes[1].scatter(datos['edad_cliente'], datos['ingresos_totales'], 
               alpha=0.6, color='purple', s=50)
axes[1].set_xlabel('Edad del Cliente')
axes[1].set_ylabel('Ingresos Totales')
axes[1].set_title('👥 Edad vs Ingresos')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 📊 Paso 8: Análisis por Categorías

Exploremos cómo se comportan nuestros datos según diferentes categorías:

In [ ]:
# Box plots para comparar distribuciones por categoría
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('📦 Distribución de Ingresos por Diferentes Categorías', fontsize=16, fontweight='bold')

# Por categoría de producto
sns.boxplot(data=datos, x='categoria', y='ingresos_totales', ax=axes[0,0])
axes[0,0].set_title('Por Categoría de Producto')
axes[0,0].tick_params(axis='x', rotation=45)

# Por región
sns.boxplot(data=datos, x='region', y='ingresos_totales', ax=axes[0,1])
axes[0,1].set_title('Por Región')

# Por método de pago
sns.boxplot(data=datos, x='metodo_pago', y='ingresos_totales', ax=axes[1,0])
axes[1,0].set_title('Por Método de Pago')
axes[1,0].tick_params(axis='x', rotation=45)

# Por grupos de edad
datos['grupo_edad'] = pd.cut(datos['edad_cliente'], 
                            bins=[0, 25, 35, 50, 100], 
                            labels=['18-25', '26-35', '36-50', '51+'])
sns.boxplot(data=datos, x='grupo_edad', y='ingresos_totales', ax=axes[1,1])
axes[1,1].set_title('Por Grupo de Edad')

for ax in axes.flat:
    ax.grid(True, alpha=0.3)
    ax.set_ylabel('Ingresos Totales')

plt.tight_layout()
plt.show()

In [ ]:
# Tabla resumen por categoría
print("📋 RESUMEN ESTADÍSTICO POR CATEGORÍA DE PRODUCTO:")
print("=" * 50)

resumen_categoria = datos.groupby('categoria').agg({
    'ingresos_totales': ['count', 'mean', 'median', 'std', 'sum'],
    'edad_cliente': 'mean',
    'cantidad': 'mean'
}).round(2)

# Simplificar nombres de columnas
resumen_categoria.columns = ['Num_Ventas', 'Ingreso_Promedio', 'Ingreso_Mediano', 
                           'Desv_Estandar', 'Ingresos_Totales', 'Edad_Promedio', 'Cantidad_Promedio']

display(resumen_categoria)

print("\n🎯 INSIGHTS CLAVE:")
categoria_mas_ingresos = resumen_categoria['Ingresos_Totales'].idxmax()
categoria_mas_ventas = resumen_categoria['Num_Ventas'].idxmax()
categoria_mayor_ticket = resumen_categoria['Ingreso_Promedio'].idxmax()

print(f"• 💰 Categoría con mayores ingresos totales: {categoria_mas_ingresos}")
print(f"• 🛍️  Categoría con más ventas: {categoria_mas_ventas}")
print(f"• 💎 Categoría con mayor ticket promedio: {categoria_mayor_ticket}")

## ⚡ Paso 9: Detección de Valores Atípicos (Outliers)

Identifiquemos valores inusuales que podrían requerir atención especial:

In [ ]:
def detectar_outliers_iqr(serie, factor=1.5):
    """
    Detecta outliers usando el método del rango intercuartil (IQR)
    """
    Q1 = serie.quantile(0.25)
    Q3 = serie.quantile(0.75)
    IQR = Q3 - Q1
    
    limite_inferior = Q1 - factor * IQR
    limite_superior = Q3 + factor * IQR
    
    outliers = serie[(serie < limite_inferior) | (serie > limite_superior)]
    
    return outliers, limite_inferior, limite_superior

# Detectar outliers en ingresos totales
print("🚨 DETECCIÓN DE VALORES ATÍPICOS:")
print("=" * 35)

variable = 'ingresos_totales'
outliers, limite_inf, limite_sup = detectar_outliers_iqr(datos[variable])

print(f"📊 Variable analizada: {variable}")
print(f"📏 Límite inferior: {limite_inf:.2f}")
print(f"📏 Límite superior: {limite_sup:.2f}")
print(f"🚨 Número de outliers: {len(outliers)} ({len(outliers)/len(datos)*100:.1f}%)")

if len(outliers) > 0:
    print(f"💰 Valor mínimo outlier: {outliers.min():.2f}")
    print(f"💰 Valor máximo outlier: {outliers.max():.2f}")

# Visualizar outliers
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Box plot con outliers marcados
bp = ax1.boxplot(datos[variable], patch_artist=True, showfliers=True)
bp['boxes'][0].set_facecolor('lightblue')
ax1.set_ylabel('Ingresos Totales')
ax1.set_title('📦 Box Plot - Detección de Outliers')
ax1.grid(True, alpha=0.3)

# Histograma con líneas de límites
ax2.hist(datos[variable], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
ax2.axvline(limite_inf, color='red', linestyle='--', label=f'Límite inf: {limite_inf:.0f}')
ax2.axvline(limite_sup, color='red', linestyle='--', label=f'Límite sup: {limite_sup:.0f}')
ax2.set_xlabel('Ingresos Totales')
ax2.set_ylabel('Frecuencia')
ax2.set_title('📊 Histograma con Límites de Outliers')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 📅 Paso 10: Análisis Temporal

Exploremos cómo cambian nuestros datos a lo largo del tiempo:

In [ ]:
# Agregar columnas de tiempo
datos['mes'] = datos['fecha'].dt.month
datos['nombre_mes'] = datos['fecha'].dt.month_name()
datos['día_semana'] = datos['fecha'].dt.day_name()

# Análisis por mes
ventas_por_mes = datos.groupby('mes').agg({
    'ingresos_totales': ['sum', 'count', 'mean']
}).round(2)

ventas_por_mes.columns = ['Ingresos_Totales', 'Num_Ventas', 'Ticket_Promedio']

# Crear gráficos temporales
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('📅 Análisis Temporal de Ventas', fontsize=16, fontweight='bold')

# Ingresos por mes
ventas_por_mes['Ingresos_Totales'].plot(kind='line', marker='o', 
                                       color='blue', linewidth=2, ax=axes[0,0])
axes[0,0].set_title('💰 Ingresos Totales por Mes')
axes[0,0].set_xlabel('Mes')
axes[0,0].set_ylabel('Ingresos Totales')
axes[0,0].grid(True, alpha=0.3)

# Número de ventas por mes
ventas_por_mes['Num_Ventas'].plot(kind='bar', color='green', alpha=0.7, ax=axes[0,1])
axes[0,1].set_title('🛍️ Número de Ventas por Mes')
axes[0,1].set_xlabel('Mes')
axes[0,1].set_ylabel('Número de Ventas')
axes[0,1].tick_params(axis='x', rotation=0)
axes[0,1].grid(True, alpha=0.3)

# Análisis por día de la semana
orden_dias = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
ventas_por_dia = datos.groupby('día_semana')['ingresos_totales'].sum().reindex(orden_dias)

ventas_por_dia.plot(kind='bar', color='orange', alpha=0.7, ax=axes[1,0])
axes[1,0].set_title('📊 Ingresos por Día de la Semana')
axes[1,0].set_xlabel('Día de la Semana')
axes[1,0].set_ylabel('Ingresos Totales')
axes[1,0].tick_params(axis='x', rotation=45)
axes[1,0].grid(True, alpha=0.3)

# Evolución diaria (muestra una semana)
datos_semana = datos[datos['fecha'].between('2023-01-01', '2023-01-07')]
ingresos_diarios = datos_semana.groupby('fecha')['ingresos_totales'].sum()

ingresos_diarios.plot(kind='line', marker='o', color='purple', linewidth=2, ax=axes[1,1])
axes[1,1].set_title('📈 Evolución Diaria (Primera Semana)')
axes[1,1].set_xlabel('Fecha')
axes[1,1].set_ylabel('Ingresos Totales')
axes[1,1].tick_params(axis='x', rotation=45)
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Estadísticas temporales
print("\n📊 ESTADÍSTICAS TEMPORALES:")
print("=" * 30)
mes_mejor = ventas_por_mes['Ingresos_Totales'].idxmax()
mes_peor = ventas_por_mes['Ingresos_Totales'].idxmin()
dia_mejor = ventas_por_dia.idxmax()
dia_peor = ventas_por_dia.idxmin()

print(f"🏆 Mejor mes: {mes_mejor} (${ventas_por_mes.loc[mes_mejor, 'Ingresos_Totales']:,.2f})")
print(f"📉 Peor mes: {mes_peor} (${ventas_por_mes.loc[mes_peor, 'Ingresos_Totales']:,.2f})")
print(f"🏆 Mejor día: {dia_mejor} (${ventas_por_dia[dia_mejor]:,.2f})")
print(f"📉 Peor día: {dia_peor} (${ventas_por_dia[dia_peor]:,.2f})")

## 🎯 Paso 11: Resumen de Insights y Conclusiones

Consolidemos los hallazgos más importantes de nuestro análisis:

In [ ]:
# Crear un resumen ejecutivo
print("📋 RESUMEN EJECUTIVO - INSIGHTS CLAVE")
print("=" * 40)

# Métricas generales
ingresos_totales = datos['ingresos_totales'].sum()
num_transacciones = len(datos)
ticket_promedio = datos['ingresos_totales'].mean()
edad_promedio = datos['edad_cliente'].mean()

print(f"💰 Ingresos totales: ${ingresos_totales:,.2f}")
print(f"🛍️  Total de transacciones: {num_transacciones:,}")
print(f"💳 Ticket promedio: ${ticket_promedio:.2f}")
print(f"👥 Edad promedio de clientes: {edad_promedio:.1f} años")

print("\n🏆 TOP PERFORMERS:")
print("-" * 20)
categoria_top = datos.groupby('categoria')['ingresos_totales'].sum().idxmax()
region_top = datos.groupby('region')['ingresos_totales'].sum().idxmax()
metodo_top = datos.groupby('metodo_pago')['ingresos_totales'].sum().idxmax()

print(f"🥇 Mejor categoría: {categoria_top}")
print(f"🥇 Mejor región: {region_top}")
print(f"🥇 Método de pago preferido: {metodo_top}")

print("\n⚠️  ÁREAS DE ATENCIÓN:")
print("-" * 25)
outliers_pct = len(outliers) / len(datos) * 100
print(f"🚨 {outliers_pct:.1f}% de transacciones son outliers")

categoria_peor = datos.groupby('categoria')['ingresos_totales'].sum().idxmin()
print(f"📉 Categoría con menores ingresos: {categoria_peor}")

print("\n💡 RECOMENDACIONES:")
print("-" * 20)
print(f"• Enfocar marketing en categoría '{categoria_top}' (mejor performance)")
print(f"• Investigar por qué '{categoria_peor}' tiene bajos ingresos")
print(f"• Optimizar estrategia para región '{region_top}'")
print(f"• Promover uso de '{metodo_top}' como método de pago")
if outliers_pct > 5:
    print("• Revisar transacciones atípicas para detectar fraudes o errores")
print("• Analizar patrones temporales para optimizar inventario")

## 📊 Paso 12: Dashboard Interactivo (Bonus)

Creemos algunas visualizaciones interactivas usando Plotly:

In [ ]:
# Gráfico de dispersión interactivo
fig = px.scatter(datos, 
                x='precio_unitario', 
                y='ingresos_totales',
                color='categoria',
                size='cantidad',
                hover_data=['edad_cliente', 'region', 'metodo_pago'],
                title='💰 Análisis Interactivo: Precio vs Ingresos por Categoría',
                labels={
                    'precio_unitario': 'Precio Unitario ($)',
                    'ingresos_totales': 'Ingresos Totales ($)',
                    'cantidad': 'Cantidad',
                    'categoria': 'Categoría'
                })

fig.update_layout(
    width=900,
    height=600,
    showlegend=True
)

fig.show()

print("💡 TIP: Pasa el cursor sobre los puntos para ver más detalles!")

In [ ]:
# Gráfico de barras interactivo por mes
ventas_mes_categoria = datos.groupby(['mes', 'categoria'])['ingresos_totales'].sum().reset_index()

fig = px.bar(ventas_mes_categoria, 
            x='mes', 
            y='ingresos_totales',
            color='categoria',
            title='📅 Ingresos Mensuales por Categoría (Interactivo)',
            labels={
                'mes': 'Mes',
                'ingresos_totales': 'Ingresos Totales ($)',
                'categoria': 'Categoría'
            })

fig.update_layout(
    width=900,
    height=500,
    showlegend=True
)

fig.show()

print("💡 TIP: Haz clic en las leyendas para mostrar/ocultar categorías!")

## 🎓 Paso 13: Próximos Pasos y Mejores Prácticas

### 🔍 Lo que hemos aprendido:

1. **Exploración inicial**: Siempre comienza entendiendo la estructura de tus datos
2. **Limpieza**: Maneja valores nulos y duplicados antes del análisis
3. **Estadísticas descriptivas**: Conoce las distribuciones de tus variables
4. **Visualización**: Las gráficas revelan patrones que los números no muestran
5. **Análisis bivariado**: Las correlaciones revelan relaciones importantes
6. **Segmentación**: Analizar por categorías revela insights específicos
7. **Outliers**: Los valores atípicos pueden ser errores o insights valiosos
8. **Análisis temporal**: Los patrones en el tiempo son cruciales para el negocio

### 🚀 Próximos pasos sugeridos:

- **Modelado predictivo**: Usar machine learning para predecir ventas
- **Segmentación de clientes**: Clustering para identificar tipos de clientes
- **Análisis de series temporales**: Forecasting de ventas futuras
- **A/B testing**: Experimentación para optimizar estrategias
- **Dashboard automatizado**: Crear reportes que se actualicen automáticamente

### 📚 Recursos para seguir aprendiendo:

- **Pandas**: Documentación oficial para manipulación de datos
- **Seaborn**: Para visualizaciones estadísticas avanzadas
- **Plotly**: Para dashboards interactivos
- **Scikit-learn**: Para machine learning
- **Streamlit**: Para crear aplicaciones web de datos

---

## 🎉 ¡Felicitaciones!

Has completado un análisis exploratorio de datos completo. Ahora tienes las herramientas para:

✅ Cargar y explorar cualquier dataset  
✅ Limpiar y preparar datos para análisis  
✅ Crear visualizaciones informativas  
✅ Identificar patrones y tendencias  
✅ Generar insights accionables para el negocio  

**¡Sigue practicando con tus propios datos!** 📊🚀